In [2]:
#load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']=(10.0,8.0)
import seaborn as sns
from scipy import stats
from scipy.stats import norm

In [3]:
#load data
#import csv
train_text = pd.read_csv("training_text",sep='\|\|',engine='python',names=['Id','Text'])
train_var = pd.read_csv("training_variants")


In [4]:
test_text = pd.read_csv("test_text",sep='\|\|',engine='python',names=['Id','Text'],skiprows=1)
test_var = pd.read_csv("test_variants")
test=test_text.join(test_var)
test.drop('ID',axis=1,inplace=True)

test.head()

,Id,Text,Gene,Variation
0,0,2. This mutation resulted in a myeloproliferat...,ACSL4,R570S
1,1,Abstract The Large Tumor Suppressor 1 (LATS1)...,NAGLU,P521L
2,2,Vascular endothelial growth factor receptor (V...,PAH,L333F
3,3,Inflammatory myofibroblastic tumor (IMT) is a ...,ING1,A148D
4,4,Abstract Retinoblastoma is a pediatric retina...,TMEM216,G77A


In [5]:
train_var.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [6]:
train=train_text.join(train_var)
train.drop('ID',axis=1,inplace=True)
train.head()

,Id,Text,Gene,Variation,Class
0,0,Cyclin-dependent kinases (CDKs) regulate a var...,FAM58A,Truncating Mutations,1
1,1,Abstract Background Non-small cell lung canc...,CBL,W802*,2
2,2,Abstract Background Non-small cell lung canc...,CBL,Q249E,2
3,3,Recent evidence has demonstrated that acquired...,CBL,N454D,3
4,4,Oncogenic mutations in the monomeric Casitas B...,CBL,L399V,4


In [7]:
train.describe(include=[np.object])

,Text,Gene,Variation
count,3321,3321,3321
unique,1921,264,2996
top,The PTEN (phosphatase and tensin homolog) phos...,BRCA1,Truncating Mutations
freq,53,264,93


In [8]:
train.drop('Id',axis=1).describe()

,Class
count,3321.000000
mean,4.365854
std,2.309781
min,1.000000
25%,2.000000
50%,4.000000
75%,7.000000
max,9.000000


In [ ]:
key=[]
for i in train['Text']:
    if i==train['Text'][4]:
        key=i
train[train['Text']==key]

In [ ]:
check=pd.DataFrame(index=train.index)
check['TextGeneVariation']=train['Text']+train['Gene']+train['Variation']
check['TextGene']=train['Text']+train['Gene']
check['TextVariation']=train['Text']+train['Variation']
check['GeneVariation']=train['Gene']+train['Variation']

In [ ]:
check.describe(include=[np.object])

In [ ]:
a=[]
for i in train['Text']:
    a.append(len(i))
check['count']=a
for key,i in check['count'].groupby(train['Class']):
    print(key,np.average(i))

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
vectorizer = TfidfVectorizer(min_df=1)

In [11]:
%%time
vec = vectorizer.fit_transform(train['Text'])

Wall time: 31 s


In [12]:
vectorizerGene = TfidfVectorizer(min_df=1)
vectorizerVariation = TfidfVectorizer(min_df=1)

In [13]:
vecGene = vectorizerGene.fit_transform(train['Gene'])
vecVariation =  vectorizerVariation.fit_transform(train['Variation'])

In [ ]:
#df_dummy = pd.get_dummies(train[['Gene','Variation']])
#count=count.join(df_dummy)
#count.head()

In [14]:
import scipy.sparse as sps
#dummies=sps.csr_matrix(df_dummy.values)

In [15]:
new=sps.hstack((vec,vecGene,vecVariation))

In [19]:
df_dummy= pd.get_dummies(train['Class'],prefix='Class')
df_dummy.head(10)

,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0
6,0,0,0,0,1,0,0,0,0
7,1,0,0,0,0,0,0,0,0
8,0,0,0,1,0,0,0,0,0
9,0,0,0,1,0,0,0,0,0


In [17]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(new,train['Class'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
result=pd.DataFrame(index=train.index,data=logistic.predict_proba(new),columns=['Class1','Class2','Class3','Class4','Class5','Class6','Class7','Class8','Class9',])
result.head(10)

,Class1,Class2,Class3,Class4,Class5,Class6,Class7,Class8,Class9
0,0.865707,0.028913,0.007426,0.015233,0.010156,0.025816,0.033830,0.005930,0.006989
1,0.049071,0.253815,0.023632,0.474829,0.088293,0.041729,0.055846,0.005940,0.006844
2,0.049071,0.253815,0.023632,0.474829,0.088293,0.041729,0.055846,0.005940,0.006844
3,0.053456,0.102271,0.098576,0.552104,0.080958,0.037616,0.059794,0.006835,0.008389
4,0.047722,0.073257,0.021582,0.656236,0.097908,0.058034,0.034005,0.005180,0.006076
5,0.047722,0.073257,0.021582,0.656236,0.097908,0.058034,0.034005,0.005180,0.006076
6,0.052040,0.079884,0.023534,0.504447,0.227454,0.063284,0.037082,0.005648,0.006626
7,0.470070,0.020286,0.017046,0.430210,0.026631,0.012378,0.014547,0.003735,0.005098
8,0.042706,0.071821,0.022685,0.678352,0.080299,0.044441,0.046102,0.006116,0.007479
9,0.045103,0.073421,0.024732,0.674860,0.070528,0.039973,0.054011,0.007706,0.009666


In [26]:
%%time
testvectorizer = TfidfVectorizer(min_df=1,vocabulary=vectorizer.get_feature_names())
testvec = testvectorizer.fit_transform(test['Text'])


Wall time: 48.8 s


In [29]:
testvectorizerGene = TfidfVectorizer(min_df=1,vocabulary=vectorizerGene.get_feature_names())
testvectorizerVariation = TfidfVectorizer(min_df=1,vocabulary=vectorizerVariation.get_feature_names())

testvecGene = testvectorizerGene.fit_transform(test['Gene'])
testvecVariation =  testvectorizerVariation.fit_transform(test['Variation'])


In [32]:
testnew=sps.hstack((testvec,testvecGene,testvecVariation))

In [40]:
result['ID']=test['Id']
result=pd.DataFrame(index=test.Id,data=logistic.predict_proba(testnew),columns=['Class1','Class2','Class3','Class4','Class5','Class6','Class7','Class8','Class9',])
result.head(10)

,Class1,Class2,Class3,Class4,Class5,Class6,Class7,Class8,Class9
Id,,,,,,,,,
0,0.067927,0.153758,0.014385,0.094591,0.033217,0.037893,0.586672,0.005318,0.006239
1,0.083383,0.101513,0.019283,0.395354,0.042818,0.054355,0.280162,0.010214,0.012918
2,0.098923,0.137811,0.023714,0.222671,0.053437,0.066576,0.376064,0.009886,0.010917
3,0.084030,0.126620,0.018183,0.138863,0.044017,0.030985,0.542694,0.006653,0.007954
4,0.106532,0.098195,0.016563,0.270327,0.048945,0.038442,0.405694,0.006947,0.008354
5,0.152089,0.169020,0.019649,0.343558,0.066537,0.052086,0.177729,0.008641,0.010692
6,0.128846,0.098375,0.034940,0.152798,0.045190,0.036874,0.488610,0.006518,0.007849
7,0.113524,0.128930,0.021517,0.311310,0.078191,0.082423,0.247990,0.007375,0.008739
8,0.155983,0.213885,0.020980,0.189932,0.046998,0.061061,0.288048,0.010226,0.012887


In [41]:
result.to_csv("Submission1.csv")